## Update Take II: The Updatening!

There are 2 questions that need re-running, so lets do it!

The 2 questions we need to re-run are:
1. "What are the '+axis+'-axis titles for each figure panel?" where "axis" = "y"  $\leftarrow$ this was never run (flagged as y-axis though in json!)
1. "'What is the aspect ratio of this figure?'" $\leftarrow$ this had an extra "persona" in the format

In [1]:
new_dir = '/Users/jnaiman/Downloads/tmp/iConf2025/tmpMultipanel/jsons/' # where to store 

orig_dir = '/Users/jnaiman/Downloads/tmp/iConf2025/visual_qa_multipanel/data_v1/data_full_v2/'


In [2]:
import json
from glob import glob
import numpy as np
from copy import deepcopy

# debug
from importlib import reload

from sys import path
path.append('../')
from utils.plot_qa_utils import init_qa_pairs

from utils.data_utils import NumpyEncoder

from utils.plot_parameters import plot_types_params
plot_params = plot_types_params.copy()

import utils.plot_qa_utils
reload(utils.plot_qa_utils)

verbose_qa = True

In [3]:
#stats = {'minimum':np.min, 'maximum':np.max, 'median':np.median, 'mean':np.mean}
stats = {'median':np.median, 'mean':np.mean}

linestyles = ['-', '--', ':'] # only use a subset of the linestyles

In [4]:
# calculate plottypes

# first, use probabilities to choose
# Prob for getting line
plot_params['line']['prob'] = 1
plot_params['histogram']['prob'] = 1
plot_params['scatter']['prob'] = 1
plot_params['contour']['prob'] = 0 # didn't do for this one

plot_types = []
for k,v in plot_params.items():
    if v['prob'] > 0:
        plot_types.append(k)

plot_types

['line', 'histogram', 'scatter']

In [5]:
files_json_qa = glob(orig_dir + '*_qa.json*')
files_json_qa[:3]

['/Users/jnaiman/Downloads/tmp/iConf2025/visual_qa_multipanel/data_v1/data_full_v2/Picture87_qa.json',
 '/Users/jnaiman/Downloads/tmp/iConf2025/visual_qa_multipanel/data_v1/data_full_v2/Picture97_qa.json',
 '/Users/jnaiman/Downloads/tmp/iConf2025/visual_qa_multipanel/data_v1/data_full_v2/Picture168_qa.json']

In [6]:
import utils.figure_level_qa_utils
reload(utils.figure_level_qa_utils)
from utils.figure_level_qa_utils import figure_qa_how_many_panels, figure_qa_plotting_style, figure_qa_colormap, figure_qa_aspect_ratio, q_plot_titles, q_plot_axis_labels, q_ticklabels, q_plot_types

import utils.general_plot_level_qa_utils
reload(utils.general_plot_level_qa_utils)
from utils.general_plot_level_qa_utils import q_errorbars_existance_lines

def figure_level_qa(data, qa_pairs, plot_types, verbose=False):
    ######## FIGURE LEVEL, L1 ########
    qa_pairs = figure_qa_how_many_panels(data, qa_pairs, verbose=verbose)
    qa_pairs = figure_qa_plotting_style(data, qa_pairs, verbose=verbose)
    qa_pairs = figure_qa_colormap(data, qa_pairs, verbose=verbose)
    qa_pairs = figure_qa_aspect_ratio(data, qa_pairs, verbose=verbose)
    # basic, plot level in general
    qa_pairs = q_plot_titles(data, qa_pairs, verbose=verbose)
    qa_pairs = q_plot_axis_labels(data, qa_pairs, axis='x', verbose=verbose)
    qa_pairs = q_plot_axis_labels(data, qa_pairs, axis='y', verbose=verbose)
    qa_pairs = q_ticklabels(data, qa_pairs, axis='x', verbose=verbose)
    qa_pairs = q_ticklabels(data, qa_pairs, axis='y', verbose=verbose)
    qa_pairs = q_plot_types(data, qa_pairs, plot_types, use_list=True, verbose=verbose)
    qa_pairs = q_plot_types(data, qa_pairs, plot_types, use_list=False, verbose=verbose)

    return qa_pairs

In [7]:
import utils.linear_plot_qa_utils
reload(utils.linear_plot_qa_utils)

from utils.linear_plot_qa_utils import q_nlines_plot_plotnums, q_stats_lines, \
    q_relationship_lines, q_linestyles_lines

def plot_level_line_qa(data, qa_pairs, iplot, stats, verbose_qa=False):
    ######### L1 #########
    qa_pairs = q_nlines_plot_plotnums(data, qa_pairs, 
                                        plot_num = iplot, 
                                        verbose=verbose_qa)
    # note: line colors don't work right now
    # qa_pairs = q_colors_lines(data, qa_pairs, 
    #                                   plot_num = iplot, 
    #                                   verbose=verbose_qa)    
    qa_pairs = q_linestyles_lines(data, qa_pairs, plot_num=iplot, 
                                    use_words=True, use_list = True,
                        linestyle_list=linestyles, verbose=verbose_qa)      
    ######### L2 #########
    # stats items
    for k,v in stats.items(): # for all stats
        qa_pairs = q_stats_lines(data, qa_pairs, stat={k:v}, 
                                    plot_num=iplot, use_words=True, 
                                    verbose=verbose_qa)
    ######## L3 #########
    for use_list in [True,False]:
        for use_nlines in [True, False]:
            qa_pairs = q_relationship_lines(data, qa_pairs, plot_num=iplot, use_words=True, 
                                        use_nlines = use_nlines, 
                                        verbose=verbose_qa, use_list=use_list)
            
    return qa_pairs

In [8]:
import utils.scatter_plot_qa_utils
reload(utils.scatter_plot_qa_utils)
from utils.scatter_plot_qa_utils import q_npoints_scatter_plot_plotnums, \
    q_stats_scatters, q_relationship_scatters

def plot_level_scatter_qa(data, qa_pairs, iplot, stats, 
                          line_list = ['random','linear','gaussian mixture model'], 
                          verbose_qa=False):
    ######### L1 #########
    qa_pairs = q_npoints_scatter_plot_plotnums(data, qa_pairs, 
                                        plot_num = iplot, 
                                        verbose=verbose_qa)
    ######### L2 #########
    # stats items
    for k,v in stats.items(): # for all stats
        for axis in ['x', 'y', 'color']:
            qa_pairs = q_stats_scatters(data, qa_pairs, stat={k:v}, 
                                        plot_num=iplot, use_words=True, 
                                        verbose=verbose_qa, axis=axis)
    # ###### L3 ######
    # type of distribution
    for axis in ['color', 'x/y']:
        qa_pairs = q_relationship_scatters(data, qa_pairs, plot_num = iplot, axis=axis,
                    return_qa=True, use_words=True, 
                    use_list=True, 
                line_list = line_list,
                verbose=verbose_qa)
        
    return qa_pairs

In [9]:
import utils.histogram_plot_qa_utils
reload(utils.histogram_plot_qa_utils)

from utils.histogram_plot_qa_utils import q_nbars_hist_plot_plotnums, q_stats_hists, q_relationship_histograms

def plot_level_histogram_qa(data, qa_pairs, iplot, stats, 
                          line_list = ['random','linear','gaussian mixture model'], 
                          verbose_qa=False):
    ######## L1 #######
    # number of bars
    qa_pairs = q_nbars_hist_plot_plotnums(data, qa_pairs, plot_num = iplot, 
                                            use_words=True, verbose=verbose_qa)
    ###### L2 #######
    # stats items
    for k,v in stats.items():
        qa_pairs = q_stats_hists(data, qa_pairs, stat={k:v}, plot_num=iplot, 
                                    use_words=True, verbose=verbose_qa)
        
    line_list = ['random','linear','gaussian mixture model']
    qa_pairs = q_relationship_histograms(data, qa_pairs, plot_num = iplot, 
            return_qa=True, use_words=True, 
            use_list=True, 
            line_list = line_list,
        verbose=verbose_qa)
    return qa_pairs

In [10]:
for json_path in files_json_qa:
    print('**************', json_path.split('/')[-1].split('.json')[0], '*************')
    with open(json_path, 'r') as f:
        qa = json.load(f)
        qa = json.loads(qa)

    # also full json
    json_path_full = json_path.replace('_qa.json', '_fullData.json')
    with open(json_path_full,'r') as f:
        data = json.load(f)
        data = json.loads(data)

    # new
    qa_pairs = init_qa_pairs()

    ##################################
    ########## L1 ###################
    #################################

    ######## FIGURE LEVEL ########
    qa_pairs = figure_level_qa(data, qa_pairs, plot_types, verbose=False)

    ###### LOOP AND PLOT-LEVEL ######
    plot_nums = []
    for k,v in data.items():
        if 'plot' in k:
            plot_nums.append(int(k.split('plot')[-1]))

    # if False:
    #     print('')
    #     print('----------- PLOT SPECIFIC ----------')
    #     print('')
    for iplot in plot_nums:
        # general plot-level questions
        ##### L2 ######
        for axis in ['x', 'y']:
            qa_pairs = q_errorbars_existance_lines(data, qa_pairs, 
                                              plot_num = iplot, 
                                              verbose=False, 
                                              axis=axis)

        if data['plot'+str(iplot)]['type'] == 'line': # line plots
            #print('----------- LINE -----------')
            qa_pairs = plot_level_line_qa(data, qa_pairs, iplot, stats, 
                                          verbose_qa=False)
        elif data['plot'+str(iplot)]['type'] == 'scatter':
            #print('----------- SCATTER -----------')
            qa_pairs = plot_level_scatter_qa(data, qa_pairs, iplot, stats, 
                          line_list = ['random','linear','gaussian mixture model'], 
                          verbose_qa=False)
        elif data['plot'+str(iplot)]['type'] == 'histogram':
            #print('----------- HISTOGRAM -----------')
            qa_pairs = plot_level_histogram_qa(data, qa_pairs, iplot, stats, 
                          line_list = ['random','linear','gaussian mixture model'], 
                          verbose_qa=False)


    json_out = deepcopy(data)
    json_out['VQA'] = deepcopy(qa_pairs)
    # dump full data
    dumped = json.dumps(json_out, cls=NumpyEncoder)
    with open(new_dir + json_path.split('/')[-1].replace('_qa.json','.json'), 'w') as f:
        json.dump(dumped, f)

    print('')
    print('****** NEXT IMAGE ********')
    print('')

************** Picture87_qa *************

****** NEXT IMAGE ********

************** Picture97_qa *************

****** NEXT IMAGE ********

************** Picture168_qa *************

****** NEXT IMAGE ********

************** Picture178_qa *************

****** NEXT IMAGE ********

************** Picture135_qa *************

****** NEXT IMAGE ********

************** Picture125_qa *************

****** NEXT IMAGE ********

************** Picture11_qa *************

****** NEXT IMAGE ********

************** Picture63_qa *************

****** NEXT IMAGE ********

************** Picture157_qa *************

****** NEXT IMAGE ********

************** Picture147_qa *************

****** NEXT IMAGE ********

************** Picture73_qa *************

****** NEXT IMAGE ********

************** Picture171_qa *************

****** NEXT IMAGE ********

************** Picture45_qa *************

****** NEXT IMAGE ********

************** Picture55_qa *************

****** NEXT IMAGE ********


In [11]:
qa_pairs['Level 1']['Figure-level questions']['plot types (list)']

{'Q': 'You are a helpful assistant that can analyze images. Please choose each plot type from the following list: [line, histogram, scatter]. What are the plot types for each panel in the figure? Please format the output as a json as {"plot types":[]}, where each element of the list refers to the plot type of a single panel. If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. ',
 'A': {'plot types': ['scatter', 'line', 'scatter', 'line', 'line']},
 'persona': 'You are a helpful assistant that can analyze images.',
 'context': 'Please choose each plot type from the following list: [line, histogram, scatter].',
 'question': 'What are the plot types for each panel in the figure?',
 'format': 'Please format the output as a json as {"plot types":[]}, where each element of the list refers to the plot type of a single panel. If there is a single plot, this should be one element in this list, and i

## Now compare new/old versions of this (i.e. pre-run)

In [12]:
old_new_dir = '/Users/jnaiman/Downloads/tmp/iConf2025/visual_qa_multipanel/jsons/'

In [13]:
old_jsons = glob(old_new_dir + '/*.json')
old_jsons[:2]

['/Users/jnaiman/Downloads/tmp/iConf2025/visual_qa_multipanel/jsons/Picture186.json',
 '/Users/jnaiman/Downloads/tmp/iConf2025/visual_qa_multipanel/jsons/Picture169.json']

In [28]:
# collect
qcollect_change = []
for ijson_path, old_json_path in enumerate(old_jsons):
    if ijson_path > 0: continue
    with open(old_json_path, 'r') as f:
        jold = json.load(f)
        jold = json.loads(jold)

    new_json_path = old_json_path.replace('/visual_qa_multipanel/', '/tmpMultipanel/')
    with open(new_json_path, 'r') as f:
        jnew = json.load(f)
        jnew = json.loads(jnew)

    jnew_vqa = jnew['VQA']
    jold_vqa = jold['VQA']
    levels = list(jnew_vqa.keys())
    for level in levels: # e.g. "Level 1", "Level 2"
        print('checking level:', level)
        qlevel = jnew_vqa[level]
        for fplevel in list(qlevel.keys()): # e.g., "Figure-level", "Plot-level"
            print('  checking level-type:', fplevel)
            qfp_new = qlevel[fplevel]
            qfp_old = jold_vqa[level][fplevel]
            # same keys
            for knew,vnew in qfp_new.items():
                #print('    checking question (new):', knew)
                if knew not in qfp_old:
                    print('Not in old key:', level, fplevel, knew)
                    import sys; sys.exit()
            for kold,vold in qfp_old.items():
                #print('    checking question (old):', kold)
                if kold not in qfp_new:
                    print('Not in new key:', level, fplevel, kold)
                    import sys; sys.exit()

            # same values
            for knew,vnew in qfp_new.items():
                #print('    checking question (new):', knew)
                if 'Q' in qfp_old[knew]: # not for a specific kind of plot
                    if qfp_old[knew] != vnew:
                        print('Old question different than new -- old:', qfp_old[knew]['Q'], ', new:', vnew['Q'])
                        qcollect_change.append({'key':knew, 'old':qfp_old[knew]['Q'], 'new':vnew['Q'], 
                                                'new format':vnew['format'], 
                                                'new question':vnew['question'], 'new context':vnew['context']})
                else: # organized by questions for specific plot numbers
                    for pn,vvold in qfp_old[knew].items():
                        if pn not in vnew:
                            print('Dont have the same plots!')
                            import sys; sys.exit()
                        if vvold != vnew[pn]:
                            print('Old question different than new -- old:', vvold['Q'], ', new:', vnew[pn]['Q'])
                            if {'key':knew, 'old':vvold['Q'], 'new':vnew[pn]['Q'], 
                                                    'new format':vnew[pn]['format'], 
                                                    'new question':vnew[pn]['question'], 'new context':vnew[pn]['context']} not in qcollect_change:
                                qcollect_change.append({'key':knew, 'old':vvold['Q'], 'new':vnew[pn]['Q'], 
                                                    'new format':vnew[pn]['format'], 
                                                    'new question':vnew[pn]['question'], 'new context':vnew[pn]['context']})  
                            else:
                                print('** repeat **')                      

                    #import sys; sys.exit()


    #import sys; sys.exit()


checking level: Level 1
  checking level-type: Figure-level questions
Old question different than new -- old: You are a helpful assistant that can analyze images.  What is the aspect ratio of this figure? You are a helpful assistant, please format the output as a json as {"aspect ratio":""} to store the aspect ratio of the plot. , new: You are a helpful assistant that can analyze images.  What is the aspect ratio of this figure? Please format the output as a json as {"aspect ratio":""} to store the aspect ratio of the plot.
Old question different than new -- old: You are a helpful assistant that can analyze images.  What are the x-axis titles for each figure panel? Please format the output as a json as {"ylabels":[]}, where the list is a list of strings of all of the y-axis titles. If there is a single plot, this should be one element in this list, and if there are multiple plots the list should be in row-major (C-style) order. If a plot does not have an y-axis title, then denote this 

In [24]:
#qfp_old[knew]

In [25]:
#vnew

In [26]:
qcollect_change

[{'key': 'aspect ratio',
  'old': 'You are a helpful assistant that can analyze images.  What is the aspect ratio of this figure? You are a helpful assistant, please format the output as a json as {"aspect ratio":""} to store the aspect ratio of the plot.',
  'new': 'You are a helpful assistant that can analyze images.  What is the aspect ratio of this figure? Please format the output as a json as {"aspect ratio":""} to store the aspect ratio of the plot.',
  'new format': 'Please format the output as a json as {"aspect ratio":""} to store the aspect ratio of the plot.',
  'new question': 'What is the aspect ratio of this figure?',
  'new context': ''},
 {'key': 'ylabels',
  'old': 'You are a helpful assistant that can analyze images.  What are the x-axis titles for each figure panel? Please format the output as a json as {"ylabels":[]}, where the list is a list of strings of all of the y-axis titles. If there is a single plot, this should be one element in this list, and if there are 

In [29]:
# save in a place
dump_file = '/Users/jnaiman/Downloads/tmp/iConf2025/tmpMultipanel/question_updates.json'
dumped = json.dumps(qcollect_change, cls=NumpyEncoder)
with open(dump_file, 'w') as f:
    json.dump(dumped, f)